In [1]:
import requests
import defs
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD']

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
code, res = fetch_candles("EUR_USD",9,"H1")

In [7]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 3943,
   'time': '2024-09-24T11:00:00.000000000Z',
   'bid': {'o': '1.11274', 'h': '1.11291', 'l': '1.11214', 'c': '1.11285'},
   'mid': {'o': '1.11282', 'h': '1.11299', 'l': '1.11221', 'c': '1.11292'},
   'ask': {'o': '1.11290', 'h': '1.11307', 'l': '1.11228', 'c': '1.11299'}},
  {'complete': True,
   'volume': 6076,
   'time': '2024-09-24T12:00:00.000000000Z',
   'bid': {'o': '1.11285', 'h': '1.11471', 'l': '1.11285', 'c': '1.11402'},
   'mid': {'o': '1.11292', 'h': '1.11478', 'l': '1.11292', 'c': '1.11410'},
   'ask': {'o': '1.11300', 'h': '1.11486', 'l': '1.11300', 'c': '1.11417'}},
  {'complete': True,
   'volume': 6434,
   'time': '2024-09-24T13:00:00.000000000Z',
   'bid': {'o': '1.11399', 'h': '1.11436', 'l': '1.11317', 'c': '1.11386'},
   'mid': {'o': '1.11407', 'h': '1.11444', 'l': '1.11324', 'c': '1.11396'},
   'ask': {'o': '1.11415', 'h': '1.11452', 'l': '1.11332', 'c': '1.11407'}},

In [8]:
code

200

In [9]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [10]:
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-24T11:00:00.000000000Z,3943,1.11282,1.11299,1.11221,1.11292,1.11274,1.11291,1.11214,1.11285,1.11290,1.11307,1.11228,1.11299
1,2024-09-24T12:00:00.000000000Z,6076,1.11292,1.11478,1.11292,1.11410,1.11285,1.11471,1.11285,1.11402,1.11300,1.11486,1.11300,1.11417
2,2024-09-24T13:00:00.000000000Z,6434,1.11407,1.11444,1.11324,1.11396,1.11399,1.11436,1.11317,1.11386,1.11415,1.11452,1.11332,1.11407
3,2024-09-24T14:00:00.000000000Z,9938,1.11396,1.11635,1.11389,1.11508,1.11387,1.11627,1.11381,1.11500,1.11406,1.11643,1.11397,1.11515
4,2024-09-24T15:00:00.000000000Z,5920,1.11509,1.11564,1.11468,1.11498,1.11501,1.11556,1.11459,1.11490,1.11517,1.11572,1.11476,1.11506
5,2024-09-24T16:00:00.000000000Z,4250,1.11498,1.11620,1.11493,1.11586,1.11490,1.11613,1.11485,1.11578,1.11507,1.11628,1.11501,1.11593
6,2024-09-24T17:00:00.000000000Z,3116,1.11585,1.11638,1.11548,1.11626,1.11577,1.11630,1.11540,1.11619,1.11593,1.11647,1.11555,1.11634
7,2024-09-24T18:00:00.000000000Z,3226,1.11628,1.11670,1.11598,1.11662,1.11620,1.11663,1.11591,1.11655,1.11636,1.11678,1.11606,1.11670


In [11]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [12]:
def create_data(pair,granularity):
    code ,json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair, granularity)

In [13]:
for p1 in our_curr:
    for p2 in our_curr:
        pair =  f"{p1}_{p2}"
        if pair in ins_df.name.unique():
          create_data(pair,"H1")

EUR_USD loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
USD_JPY loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
USD_CHF loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
USD_CAD loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
GBP_USD loaded 3999 candles from 2024-02-05T03:00:00.000000000Z to 2024-09-24T18:00:00.000000000Z
GBP_JPY loaded 3999 

In [14]:
ins_df.shape[0]

68

In [15]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,USD_CNH,CURRENCY,USD/CNH,-4,0.05
1,NZD_CAD,CURRENCY,NZD/CAD,-4,0.03
2,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
3,SGD_CHF,CURRENCY,SGD/CHF,-4,0.05
4,EUR_CZK,CURRENCY,EUR/CZK,-4,0.05
...,...,...,...,...,...
63,USD_SEK,CURRENCY,USD/SEK,-4,0.03
64,GBP_SGD,CURRENCY,GBP/SGD,-4,0.05
65,EUR_PLN,CURRENCY,EUR/PLN,-4,0.05
66,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1


In [16]:
ins_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [17]:
ins_df.columns


Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [18]:
ins_df.name

0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object

In [19]:
ins_df.name.unique

<bound method Series.unique of 0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object>